<a href="https://colab.research.google.com/github/JacobJS56/FootballRaterService/blob/main/Football_Webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [1]:
import requests
import time
import re
import json
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/gdrive')

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Mounted at /content/gdrive


In [2]:
URL_PREFIX = "https://www.footballdatabase.eu"

# Teams

In [3]:
URL = URL_PREFIX + "/en/competition/overall/14896-premier_league/2021-2022"
page = requests.get(URL)
league = "Premier League"
main_soup = BeautifulSoup(page.content, "html.parser")

In [4]:
teamList = main_soup.find_all("div", class_="row")
teamList = main_soup.find_all("div", class_="module ranking fullsize")
teamList = main_soup.find_all("td", class_="club")
teamList = teamList[0:20]

In [5]:
team_dict = dict()

for team in teamList:
  team_name = team.text.strip()
  team = team.find("a", class_="")
  team_url = team.get('href')
  team_dict[team_name] = team_url

# Players

Gets all players from all teams

In [6]:
teams_dict_withplayers = dict()

for key in team_dict:
 # time.sleep(25)
  hashedKey = str(abs(hash(key)) % (10 ** 8))
  URL = URL_PREFIX + team_dict[key]
  page = requests.get(URL)
  team_soup = BeautifulSoup(page.content, "html.parser")

  listOfPlayers = []
  teamD = dict()
  tsuc=False
  ssuc=False
  msuc=False

  # league
  teamD["league"] = league

  # logo
  try:
    logo = team_soup.find("div", class_="clublogo")
    for l in logo:
      with open('/content/gdrive/My Drive/football/logos/' + hashedKey + '-logo.png', 'wb') as f:
        f.write(requests.get(URL_PREFIX + str(l["src"])).content)
      time.sleep(25)
    tsuc=True
  except:
    print("Couldn't get Logo for: %s", key)

  # stadium
  try:
    stadium = team_soup.find("div", class_="info")
    stadium = str(stadium).split("<div>")[5]
    stadium = re.sub("</div>", "", str(stadium))
    stadium = re.sub("(.*)span>", "", str(stadium))
    teamD["stadium"] = stadium
    ssuc=True
  except:
    print("Couldn't get Stadium for: %s", key)

  # manager
  try:
    manager = team_soup.find("div", class_="info-manager")
    manager.find("p", class_="manager")
    manager = re.sub("Manager(.*)", "", str(manager.text.strip()))
    manager = manager.split("\n")[0]
    teamD["manager"] = manager
    msuc=True
  except:
    print("Couldn't get Manager for: %s", key)

  # find the logos shareable link
  for d in drive.ListFile({'q': "'1MSknsWLdfdb7yFZ906uT4JZOA4-L5Ni6' in parents"}).GetList():
    if(d['originalFilename'] == hashedKey + '-logo.png'):
      print(d['originalFilename'])
      teamD["logo"] = d['alternateLink']

  # players
  playerList = team_soup.find_all("div", class_="headband-player")
  for player in playerList:
    player_name = player.find("h3", class_="name")
    listOfPlayers.append(player_name.text.strip())
  teamD["playerList"] = listOfPlayers

  teams_dict_withplayers[key] = teamD

  if(ssuc and msuc and tsuc):
    print("Succesful for: %s", key)

19621764-logo.png
Succesful for: %s Arsenal
58010305-logo.png
Succesful for: %s Everton
18164497-logo.png
Succesful for: %s Brentford
91317971-logo.png
Succesful for: %s Leeds United
55785547-logo.png
Succesful for: %s Brighton & Hove
63252072-logo.png
Succesful for: %s West Ham
54216007-logo.png
Succesful for: %s Burnley
65997850-logo.png
Succesful for: %s Newcastle United
40658250-logo.png
Succesful for: %s Chelsea
40701511-logo.png
Succesful for: %s Watford
36931136-logo.png
Succesful for: %s Crystal Palace
8383660-logo.png
Succesful for: %s Man United
41958970-logo.png
Succesful for: %s Leicester
69611650-logo.png
Succesful for: %s Southampton
96723161-logo.png
Succesful for: %s Liverpool
19459540-logo.png
Succesful for: %s Wolverhampton
43595756-logo.png
Succesful for: %s Manchester City
14234953-logo.png
Succesful for: %s Aston Villa
29524678-logo.png
Succesful for: %s Norwich City
94292349-logo.png
Succesful for: %s Tottenham


In [7]:
print(json.dumps(teams_dict_withplayers, indent=4))

{
    "Arsenal": {
        "league": "Premier League",
        "stadium": "Emirates Stadium",
        "manager": "Mikel Arteta",
        "logo": "https://drive.google.com/file/d/1PVkbH-FW2s10tGhejHZmubMxX3Obo7ep/view?usp=drivesdk",
        "playerList": [
            "Runar Runarsson",
            "Aaron Ramsdale",
            "Karl Hein",
            "Arthur Okonkwo",
            "Bernd Leno",
            "Pablo Mar\u00ed",
            "Takehiro Tomiyasu",
            "Ben White",
            "Mazeed Ogungbo",
            "Zach Awe",
            "Alex Kirk",
            "Rob Holding",
            "Gabriel",
            "Ryan Alebiosu",
            "C\u00e9dric Soares",
            "H\u00e9ctor Beller\u00edn",
            "Calum Chambers",
            "Nuno Tavares",
            "Sead Kolasinac",
            "Kieran Tierney",
            "Albert Lokonga",
            "Mohamed El Neny",
            "Thomas Partey",
            "Zak Swanson",
            "Miguel Azeez",
            "Sala

In [8]:
jsonString = json.dumps(teams_dict_withplayers, indent=4)
with open('/content/gdrive/My Drive/football/' + league + '.json', 'w') as f:
  f.write(jsonString)

In [9]:
for d in drive.ListFile({'q': "'1MSknsWLdfdb7yFZ906uT4JZOA4-L5Ni6' in parents"}).GetList():
  print(d)

GoogleDriveFile({'kind': 'drive#file', 'id': '1-ZzlLYNP1_j6pRUiv8OipZNu7wgkGtbw', 'etag': '"MTY1NjQ1NjE0MzAwMA"', 'selfLink': 'https://www.googleapis.com/drive/v2/files/1-ZzlLYNP1_j6pRUiv8OipZNu7wgkGtbw', 'webContentLink': 'https://drive.google.com/uc?id=1-ZzlLYNP1_j6pRUiv8OipZNu7wgkGtbw&export=download', 'alternateLink': 'https://drive.google.com/file/d/1-ZzlLYNP1_j6pRUiv8OipZNu7wgkGtbw/view?usp=drivesdk', 'embedLink': 'https://drive.google.com/file/d/1-ZzlLYNP1_j6pRUiv8OipZNu7wgkGtbw/preview?usp=drivesdk', 'iconLink': 'https://drive-thirdparty.googleusercontent.com/16/type/image/webp', 'thumbnailLink': 'https://lh6.googleusercontent.com/3ADf1uJetRVt7aEOGa8RfBNIuR_5QSrlr2E4zZZnxdorL0057wjqCnNp7eJeLt7LrG0i0bis-HTpydY=s220', 'title': '94292349-logo.png', 'mimeType': 'image/webp', 'labels': {'starred': False, 'hidden': False, 'trashed': False, 'restricted': False, 'viewed': True}, 'copyRequiresWriterPermission': False, 'createdDate': '2022-06-28T22:42:23.000Z', 'modifiedDate': '2022-06-2